# Konstrukcja sieci neuronowej

Rozważmy sieć neuronową składającą się z warstw $1,...,L$

Gdzie:
- $n_l - $ liczba neuronów w warstwie l
- $a^l \in \mathbb{R}^{n_l}$ - wektor aktywacji warstwy $l$
- $z^l \in \mathbb{R}^{n_l}$ - wektor sum ważonych przed aktywacją
- $W^l \in \mathbb{R}^{n_l \times n_{l-1}}$ - macierz wag między warstwą $l-1$ i $l$
- $b^l \in \mathbb{R}^{n_l}$ - wektor biasów warstwy $l$

## Feedforward (obliczanie wyjścia sieci)

1.1 **Suma ważona**
    $z^l = W^la^{l-1}+b^l$

1.2 **Funkcja aktywacji (sigmoid):**
    $a^l = \sigma(z^l)$
    $\sigma(z) = \frac{1}{1+e^{-z}}$

Wyjściem sieci jest wektor :
    $\hat{y} = a^L$

## Funkcja kosztu

Dla pojedynczej próbki (x,y)  definiujemy koszt kwadratowy:

$ C = \frac{1}{2}{||a^L-y||}^2 $

Pochodna po aktywacji w warstwie wyjściowej

$ \frac{\partial{C}}{\partial{a^L}} = a^L - y $ 

## Backpropagation

Celem backpropagation jest policzyć 
$\frac{\partial{C}}{\partial{W^l}},\frac{\partial{C}}{\partial{b^l}}$

dla wszystkich warstw $l = 1,2,3...,L$

Definiujemy pomocniczą wielkość:

${\delta}^l = \frac{\partial{C}}{\partial{z^l}}$

nazywaną błędem warstwy l.

## Błąd w warstwie wyjściowej L

Wykorzystując wzory:

${\delta}^l = ({\frac{\partial{C}}{\partial{z^l}}})\odot {\sigma}'(z^L)$

gdzie :

$\sigma'(z) = (a^L-y)\odot{\sigma'(z^L)}$

## Błąd w warstwach ukrytych

Dla każdej warstwy $l=L-1, L-2,...,1$:


$\delta^l = (({W^{l+1}})^T{\delta}^{l+1})\odot{\sigma'(z^l)}$

Interpretacja :
- błędy warstwy następnej wracają wstecz przez transponowaną macierz wag
- pochodna sigmoidu przeskalowuje wpływ na aktualną warstwę


## Gradient wag i biasów

**Gradient biasów**

$\frac{\partial{C}}{\partial{b^l}} = \delta^l$

**Gradient wag**

$\frac{\partial{C}}{\partial{W^l}} = {\delta^l}(a^{l-1})^T$

Interpretacja:

- błąd neuronu $\theta^l$
- pomnożony przez aktywację neurony z  warstwy wcześniejszej
- daje wpływ każdej wagi na błąd

## Stochastic Gradient Descent (SGD)

Uczenie polega na aktualizacji wag i biasów:

$W^l \leftarrow W^l - \eta * \frac{\partial{C}}{\partial{W^l}}$

$b^l \leftarrow b^l - \eta * \frac{\partial{C}}{\partial{b^l}}$

gdzie :

- $\eta$ - learning rate

## Praktyka

W praktyce liczymy gradienty nie dla pojedynczej próbki, ale dla mibi-batcha

Niech mini-batch ma m elementów:

$B = {[(x_1,y_1),(x_2,y_2),...,(x_m,y_m)]}$

Gradienty sumujemy:

$$
\nabla_{W^l} C_B = \sum_{j=1}^{m} \frac{\partial C_{x_j}}{\partial W^l}
$$

$$
\nabla_{b^l} C_B = \sum_{j=1}^{m} \frac{\partial C_{x_j}}{\partial b^l}
$$

I aktualizacja wygląda tak:

$$
W^l \leftarrow W^l - \frac{\eta}{m}\,\nabla_{W^l} C_B
$$

$$
b^l \leftarrow b^l - \frac{\eta}{m}\,\nabla_{b^l} C_B
$$


In [ ]:
# fro matrxi calculations
import numpy as np

# for random test data generation
import random


class Network(object):
    def __init__(self, sizes):
        """Initializes our Network object

        Initializes Network object with given sizs of hidden layers

        Args:
            sizes (list of ints)  - lengths of hiddens layers
        """
        # number of hidden layers
        self.num_layers = len(sizes)

        # random generated biases
        self.biases = [np.random.randn(y, 1)
                       for y in sizes[1:]]
        
        # random generated weights
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
           
    def feedforward(self, a):
        """ FeedForward func 

        It is responsible for calculating actiations for 
        next neureons in our neural network

        Args:
            self (Network) : Object
            a (float list) : Outputs fromm last layers
        """
        for b, w in zip(self.biases, self.weights):

            # calculating with sigmoid function
            a = sigmoid(np.dot(w, a)+b)

        return a
        
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Stochastic Gradient Descent
        
        Finds local minima for weights and biases of our neural network 
        using Stochastic Gradient Descent Algorithm

        Args:
            self (Network) : Object
            training_data (list of pairs) : Image and number on image
            epochs (int) :  Number of learning steps of our network
            mini_batch_size (int) :  Length of batches that training data should be divied into
            eta (float) : Length of our SGD step 
            test_data (list of pairs) : Image and number on image
        
        """

        if test_data:
            n_test = len(test_data)

        n = len(training_data)

        for j in range(epochs):

            #random training data shuffle
            random.shuffle(training_data)

            # rendering list of mini batches
            mini_batches = [
                training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size )
                ] 
            
            # teaching our nneural network on with every batch
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)

            # evaluating performance of our neural network
            # after previous learning step
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data),n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """ Updates neural network with single batch

            Executes single SGD step on single mini_batch

            Args:
                mini_batch (list of pairs) : Image and number on image
                eta (float) : length of SGD step
        """

        # dla każdej warstwy w każdej sieci są biases
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # dla każdej warstwy w każdej sieci są weights
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            # propagacja wsteczna

            delta_nabla_b, delta_nabla_w = self.backprop(x, y)


            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]


            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        # update biasów i wag

        self.weights = [w - (eta/len(mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        
        self.biases = [b - (eta/len(mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]
    

    def backprop(self, x, y):

        # tworzymy wetkroy zer dla b 
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # tworzymy wetkroy zer dla b 
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # aktywacje
        activation = x
        activations = [x]
        zs = []

        for b, w in zip(self.biases, self.weights):
            # przemnazamy aktywacje prze wagi i dodajemy bias
            z = np.dot(w,activation)+b
            zs.append(z)

            # przepuszczamy przez funckje sigmoid
            activation = sigmoid(z)
            activations.append(activation)

        
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1] )

        nabla_b[-1] = delta

        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):

            z = zs[-l]

            sp = sigmoid_prime(z)

            delta = np.dot(self.weights[-l+1].transpose(),delta) * sp

            nabla_b[-l] = delta
            
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) 
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    # funckja aktywacyjna
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
    
# funcjja sigmoid
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# jakis dziwny sigmoid, nie rozumiem go
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))
